In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center><img src="https://static1.squarespace.com/static/5eddaaf244c33b340e4f21bd/t/5f61a0d12e7ca571c592893e/1602469387294/?format=1500w"></center>

<h2 style="color:red"><center>Riiid Answer Correctness Prediction</center><h2>
<h2 style="color:orange"><center>Riiid AIEd Challenge 2020: Open an New Era of Education</center><h2>    

<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Table of Contents</h3>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#about-comp" role="tab" aria-controls="profile">About the Competition<span class="badge badge-primary badge-pill">1</span></a>   
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#explore-data" role="tab" aria-controls="profile">Exploring the Data<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#train-data" role="tab" aria-controls="messages">Explore Train data<span class="badge badge-primary badge-pill">3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#quest-data" role="tab" aria-controls="messages">Explore Questions Data<span class="badge badge-primary badge-pill">4</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#lect-data" role="tab" aria-controls="settings">Explore Lectures Data<span class="badge badge-primary badge-pill">5</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#ss-sub" role="tab" aria-controls="settings">Sample Test and Submission Data<span class="badge badge-primary badge-pill">6</span></a> 

# <a id="about-comp">1. About the Competition</a>

Think back to your favorite teacher. They motivated and inspired you to learn. And they knew your strengths and weaknesses. The lessons they taught were based on your ability. For example, teachers would make sure you understood algebra before advancing to calculus. Yet, many students don’t have access to personalized learning. In a world full of information, data scientists like us can help. Machine learning can offer a path to success for young people around the world, and you are invited to be part of this mission.

In 2018, 260 million children weren't attending school. At the same time, more than half of these young students didn't meet minimum reading and math standards. Education was already in a tough place when COVID-19 forced most countries to temporarily close schools. This further delayed learning opportunities and intellectual development. The equity gaps in every country could grow wider. We need to re-think the current education system in terms of attendance, engagement, and individualized attention.

In this competition, your challenge is to create algorithms for "Knowledge Tracing," the modeling of student knowledge over time. The goal is to accurately predict how students will perform on future interactions. You will pair your machine learning skills using Riiid’s EdNet data.

<p style="color:orange">Prize-winning teams will be invited to present their models at the AAAI-2021 Workshop on AI Education - Imagining Post-COVID Education with AI - in February 2021. All competition participants are welcome to submit their write-ups to the workshop.</p>

# 1.1. Some Points to keep in mind

- We have to use the official api to make our submissions.
- Refer to the [starter notebook](https://www.kaggle.com/sohier/competition-api-detailed-introduction) for an example of how to complete a submission. 
- We should not try to submit anything for the rows that contain lectures.
- The API provides user interactions groups in the order in which they occurred. 
- Each group will contain interactions from many different users, but no more than one task_container_id of questions from any single user. 
- Each group has between 1 and 1000 users.

**Expect to see roughly 2.5 million questions in the hidden test set.**

- The API will load up to 1 GB of the test set data in memory after initialization. The initialization step (env.iter_test()) will require meaningfully more memory than that; we recommend you do not load your model until after making that call. The API will also consume roughly 15 minutes of runtime for loading and serving the data. Hence, after making a submission, go and grab a cup of coffee.

- The API loads the data using the types specified above (int32 for user_id, int8 for content_type_id, etc).

- A baseline prediction using Keras and 5Fold Cross validation can be reffered [here](https://www.kaggle.com/mrutyunjaybiswal/riiid-neural-nets-starter-baseline-in-5f-cv)

# 1.2. Updates, Corrections and Clarifications

- Lecture tags will match the tags in `questions.csv`.
- The hidden test set contains new users but not new questions.
- The train/test data is complete, in the sense that there are no missing interactions in the union of train and test data. It remains possible that some questions weren't logged due to other issues that all datasets of mobile users are susceptible to, such as if a user lost their connection mid-question.
- The test data follows chronologically after the train data. The test iterations give interactions of users chronologically.

> Evaluation Metrics: Area under the ROC Curve between the predicted probability and observed target

And ofcoures yes, we gotta maximize our score. :)

# 2. <a id="explore-data">Exploring the Data<a>

We have got really huge amount of data, and the hosts made it even interesting by testing our model using the kaggle time series API. More about the time series API can be found [here](https://www.kaggle.com/c/riiid-test-answer-prediction/data).

**CSV Files**

1. `train.csv`
2. `questions.csv`: Meta data for the questions posted to users
3. `lectures.csv`: metadata for the lectures watched by users as they progress in their education
4. `example_test.csv`: Sample test data as it will be delivered by timeseries API. Almost same as `train.csv`
5. `example_sample_submission.csv`

Let's explore one by one.

<h2 style="color:DarkGoldenRod">Tip #1</h2>
 
Let's use datatable to load the complete data just for the shake of seeing the wall time. As we will proceed, we will use only $0.01\%$ of whole data, i.e. around **1M data points**.

Here is an awesome tutorial by World's third quadruple grandmaster Vopani:
Notebook: https://www.kaggle.com/rohanrao/tutorial-on-reading-large-datasets

To use datatable, you can do any of the following steps:
1. Add datatable as a dataset to your kernel. Here is the dataset link by none other than Vopani: https://www.kaggle.com/rohanrao/python-datatable and then follow the steps in next code block and run it.
2. Another way, if you fail to do the above step, then just fork this notebook and see how's that been performed ;)

In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1

In [ ]:
import os
import gc
import random
import numpy as np
import pandas as pd
import datatable as dt
from collections import Counter, defaultdict

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import seaborn as sns
import matplotlib.pyplot as plt

import joblib

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

colorMap = sns.light_palette("orange", as_cmap=True)

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

_ = np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# Helper functions
def distribution_plot(target_df, column, color, n=40):
    df = target_df[column].value_counts().reset_index()
    df.columns = [column,'count']
    df[column] = df[column].astype(str) + '-'
    df = df.sort_values(['count'],ascending=False)

    plt.figure(figsize=(15,10))
    plt.subplot(121)
    sns.distplot(df['count'],color=color)
    plt.title(f"{column} distribution plot")

    plt.subplot(122)
    sns.barplot(x='count',y=column,data=df[:n],orient='h')
    plt.title(f"{column} count plot")
    plt.show()

# 3. <a id="train-data">Explore Train data</a>

Here is the significance of each column.

- `row_id`: (int64) ID code for the row.

- `timestamp`: (int64) the time between this user interaction and the first event from that user.

- `user_id`: (int32) ID code for the user.

- `content_id`: (int16) ID code for the user interaction

- `content_type_id`: (int8)(Binary category) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

- `task_container_id`: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id. Monotonically increasing for each user.

- `user_answer`: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

- `answered_correctly`: (int8) if the user responded correctly. Read -1 as null, for lectures.

- `prior_question_elapsed_time`: (float32) How long it took a user to answer their previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Note that the time is the total time a user took to solve all the questions in the previous bundle.

- `prior_question_had_explanation`: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

Let's read the whole train dataframe using our weapon datatable. Before that, let's decide the dtypes to load in an optimized way. For better understanding, do refer to the organizer's notebooks.

In [ ]:
data_types_dict = {
    'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'content_type_id':'int8', 
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool'
}

target = 'answered_correctly'

In [ ]:
%%time

train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()
train_df.head()

In [ ]:
train_df.nunique().to_frame().rename(columns={0:"Unique_values"}).style.background_gradient(cmap=colorMap)

<h2 style="color:DarkGoldenRod">Takeaway #1</h2>

<h3 style="font:bold">Continuous Features</h3>

<ul>
    <li style="color:coral">Time Features</li>
        <ul>
            <li style="color:darkviolet">timestamp</li><p>Time between the user ineraction and the first event from that user.</p>
            <li style="color:darkviolet">prior_questions_elapsed_time</li><p>Avg time a user took to solve each question in the previous bundle. For Ex: if there are three questions in a bundle, they share the same <em>prior_question_elapsed_time</em> and the value is the total time the user took to solve all three questions divided by three. <b>Null</b> for user's first questions bundle or lecture.</p>
        </ul>
    <li style="color:coral">ID Features</li>
        <ul>
            <li style="color:darkviolet">content_id</li>
            <p>13320 Unique values. ID code for the user interaction. Also refers to the <em>question_id</em> in the <b>questions.csv</b> file.</p>
            <li style="color:darkviolet">task_container_id</li><p>7740 unique values. ID code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a <em>task_container_id</em>. <b>Monotonically increasing for each user</b>.</p>
        </ul>
</ul>

<h3 style="font:bold">Categorical Features</h3>

<ul>
    <li style="color:darkviolet">content_type_id</li>
        <ul>
            <li>Binary Value</li><p>0 corresponds to <em>content_id</em> is the <em>question_id</em> of <b>questions.csv</b> and 1 corresponds to <em>content_id</em> also but it is the <em>lecture_id</em> of <b>lectures.csv</b>.</p>
        </ul>
    <li style="color:darkviolet">user_answer</li>
        <ul>
            <li>Contains 4 values</li>
            <p>User's answer to the questions. Values are: <em>0, 1, 2, 3, and -1</em>. -1 corresponds to Null value for lectures (not questions type content, i.e. valid for content_type_id == 1 only).</p>
        </ul>
    <li style="color:darkviolet">prior_question_had_explanation</li>
        <ul>
            <li> Boolean feature, contains True, False and NA.
            <p>Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between.</p>
            <p>The value is shared across a single question bundle.</p>
            <p> Null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.</p>
        </ul>
    <li style="color:darkviolet">user_id</li>
    <p>What about <em>user-id</em>? low-cardinal feature? High-cardinal feature? Should we drop? How can we relate it analytically to our target?</p>
</ul>
<h3 style="font:bold; color:red">Target</h3>
    <li style="color:orangered">answered_correctly</li>
    <ul>
        <li>Binary Value i.e 1 or 0.</li>
        <li>-1 as Null, and it implies the content is a lecture => content_type_id == 1.</li>
        <li> Ofc, Binary classification and Metrics is AUC_ROC.</li>
    </ul>

Let's understand the data more and try to find some interesting patterns.
<p style="color:red">If you find the kernel interesting, consider upvoting it and open a discussion thread in the comments section for further improvements. Thanks in advance :).</p>

*Let's perform our analysis on 0.01 percent of whole data to save time as well as memory.*

In [ ]:
train = train_df.sample(frac=0.01)
print("Length of our sample dataset: ", len(train))
display(train)
# delete original dataset
del train_df
gc.collect()

# <h2 style="color:dodgerblue">3.1. Missing/Nan Values</h2>
Target Columns: <p style="color:midnightblue">user_answer, prior_question_had_explanation, prior_question_elapsed_time, answered_correctly</p>

In [ ]:
train.isna().sum().to_frame().rename(columns={0:"Nan_values"}).style.background_gradient(cmap=colorMap)

As mentioned in the data description, Null/Nan values in `prior_question_elapsed_time` and `prior_question_had_explanation` corresponds to the user's first questions bundle or first lecture. For `user_answer` and `answered_correctly`, -1 represents NULL values where the content type is a lecture i.e `content_type_id == 1`. Let's check it out, how many questions and lectures contents are there in this 1M chunk.

In [ ]:
train['content_type_id'].value_counts().rename(index={0:"Questions", 1:"Lectures"}).to_frame().rename(columns={0:"Counts"}).style.background_gradient(cmap=colorMap)

We can see that there are 19399 lecture contents in the train set and it has same numbers of Null values in `user_answer` and `answered_correctly` column. As these features/data points won't be there in the testing, we will drop that, i.e drop the rows having `content_type_id == 1`. Let's check the percentage of **to-be-dropped** rows in our dataset.

In [ ]:
correct_counts = train['answered_correctly'].value_counts().rename(index={0:"No", 1:"Yes", -1:"NaN"}).reset_index()
correct_counts.columns = ['answered_correctly', 'percentage']
correct_counts.loc[:, "percentage"] /= len(train)

user_answer_counts = train['user_answer'].value_counts().rename(index={-1:"NaN"}).reset_index()
user_answer_counts.columns = ['User_answer', 'percentage']
user_answer_counts.loc[:, "percentage"] /= len(train)

def func(pct, allvalues):
    absolute = int(pct /100.*np.sum(allvalues))
    return "{:.1f}%".format(pct)
    pass

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 20))
wegdes, texts, autotexts = ax1.pie(correct_counts['percentage'],
                                  autopct=lambda pct: func(pct, correct_counts['percentage']),
                                  explode=(0.05, 0.05, 0.05),
                                  labels=correct_counts['answered_correctly'],
                                  shadow=True,
                                  startangle=45,
                                  wedgeprops={"linewidth":1, "edgecolor":"black"},
                                  textprops= dict(color="black"))
ax1.legend(wegdes, correct_counts['answered_correctly'],
          title="Answered Correctly",
          loc="center",
          bbox_to_anchor=(1, 0, 0, 0))
plt.setp(autotexts, size=14, weight="bold")
ax1.set_title("Answered Correctly Percentage")

wegdes, texts, autotexts = ax2.pie(user_answer_counts['percentage'],
                                  autopct=lambda pct: func(pct, user_answer_counts['percentage']),
                                  explode=(0.05, 0.05, 0.05, 0.05, 0.05),
                                  labels=user_answer_counts['User_answer'],
                                  shadow=True,
                                  startangle=45,
                                  wedgeprops={"linewidth":1, "edgecolor":"black"},
                                  textprops= dict(color="black"))
ax2.legend(wegdes, user_answer_counts['User_answer'],
          title="User Answered",
          loc="center",
          bbox_to_anchor=(1, 0, 0, 0))
plt.setp(autotexts, size=14, weight="bold")
ax2.set_title("User Answers Percentage")

fig.show()

In [ ]:
# free up spaces
del user_answer_counts, correct_counts
gc.collect()

$2\%$ is very insignificant, so let's drop the rows and check how many NaN values remaines in the columns of `prior_question_had_explanation` and `prior_question_elapsed_time`.

In [ ]:
drop_indexes = train[train['answered_correctly']==-1].index.to_list()
drop_data = train.loc[drop_indexes, :]
retained_train = train.drop(index=drop_indexes).reset_index(drop=True)

print("Dropped datapoints (datapoints corresponding to lectures)")
display(drop_data)
print("Retained datapoints (datapoints corresponding to questions)")
display(retained_train)

Let's check for remaining NaN values.

In [ ]:
retained_train.isna().sum().to_frame().rename(columns={0:"Nan_values"}).style.background_gradient(cmap=colorMap)

Okay, so the remaining NaN values are because of either first question bundle or lecture for that corresponding user. We will see how can impute these values later in our modelling notebook.

# <h2 style="color:dodgerblue">3.2. More on Continuous Features: Trends? exceptions? outliers?</h2>

Target Columns: <p style="color:midnightblue">timestamp, prior_question_elapsed_time, task_container_id, content_id, user_id, user_answer</p>

In [ ]:
plt.figure(figsize=(36, 20))
sns.set_style('dark')

chunk_data = retained_train.copy()
chunk_data = chunk_data.sort_values(by=['timestamp'])
chunk_data = chunk_data.drop_duplicates('timestamp')
# we have 7,49,458 data points here, we will analyse into three pieces, having 2500 data points each from starting, middle and ending

chunk1 = chunk_data.head(5000)
plt.subplot(3, 1, 1)
sns.pointplot(x=chunk1['timestamp'],
             y=chunk1['prior_question_had_explanation'],
             hue=chunk1['answered_correctly'],
             linestyle='--', color='yellow', markers='x')
plt.xticks([])
plt.yticks([0, 1])

chunk2 = chunk_data[349500:350500]
plt.subplot(3, 1, 2)

sns.pointplot(x=chunk2['timestamp'],
             y=chunk2['prior_question_had_explanation'],
             hue=chunk2['answered_correctly'],
             linestyle='--', color='orange', markers='x')
plt.xticks([])
plt.yticks([0, 1])

chunk3 = chunk_data.tail(1000)
plt.subplot(3, 1, 3)
sns.pointplot(x=chunk3['timestamp'],
             y=chunk3['prior_question_had_explanation'],
             hue=chunk3['answered_correctly'],
             linestyle='--', color='red', markers='x')
plt.xticks([])
plt.yticks([0, 1])

plt.show()

Things to look upto:

- With increasing timestamp i.e. time, the users become more efficient with answering correctly. This indicates a sequence model might give/beat state of the art/bench mark result ;)
- In initial times, there are a very few explanations to the questions, yeah, its not a finding. It's a confirmation of the fact that user's first question bundle or lecture had no prior explanation.

# <h2 style="color:dodgerblue">3.3. Timestamps. What do you have pal?</h2>

- This is gonna play an important role in transformer models. 

In [ ]:
# let's explore more about timestamp
# as it is known that, the timestamps are given in an incremental manner, i.e. non-decreasing, let's see user interaction
plt.figure(figsize=(20, 8))
sns.set_style("dark")
retained_train['timestamp'].hist(bins=50, color="blue", legend=True)
plt.show()

In [ ]:
retained_train[retained_train['timestamp']==0]

Well, well, I see something. There are 3824 unique user IDs even after removing lecture rows from the data. But here we can see that there are 3847 rows having `timestamp` value `0`. Let's find out the users having started an event or interaction at 0 ms.

In [ ]:
init_timestamp_df = retained_train[retained_train['timestamp']==0]
user_names = init_timestamp_df['user_id'].to_list()
user_dictionary = Counter(user_names)

multiTaskers = []
for key, val in user_dictionary.items():
    if val >1 :
        multiTaskers.append(key)
print(f"There are {len(multiTaskers)} multi-taskers in this 1M chunk.")       

for ui in multiTaskers:
    display(init_timestamp_df[init_timestamp_df['user_id']==ui])

In [ ]:
# free up spaces
del init_timestamp_df
retained_train.drop(columns=['content_type_id'], inplace=True)
gc.collect()
display(retained_train)

Key findings:

- There are 8 multitasking users given `timestamp==0`. Though it might differ as per the sampling, because the sampling is done randomly.
- Each user has same `prior_question_elapsed_time` and `task_container_id`.
- Each user has neighboring `content_id`, not necessarily incremental.
- Same users also have instances of multitasking more than once.

In [ ]:
def annotate(data, **kws):
    n = len(data)
    ax = plt.gca()
    ax.text(.7, .8, f"N = {n}", transform=ax.transAxes)
    pass
sns.set_style('whitegrid')
g = sns.FacetGrid(retained_train, col="user_answer", height=4.5, aspect=0.8)
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)

g.map(sns.scatterplot, 'timestamp', 'prior_question_elapsed_time',alpha=1, edgecolor='black',color='red')
g.map_dataframe(annotate)
plt.show()

Well, the count shows users have answered `2` lesser than the rest.

In [ ]:
fig = make_subplots(rows=2, cols=2)

traces = [
    go.Bar(
        x=[-1, 0, 1], 
        y=[
            len(retained_train[(retained_train['user_answer']==item) & (retained_train['answered_correctly'] == -1)]),
            len(retained_train[(retained_train['user_answer']==item) & (retained_train['answered_correctly'] == 0)]),
            len(retained_train[(retained_train['user_answer']==item) & (retained_train['answered_correctly'] == 1)])
        ], 
        name='Option: ' + str(item),
        text = [
            str(round(100 * len(retained_train[(retained_train['user_answer']==item) & (retained_train['answered_correctly'] == -1)]) / len(retained_train[(retained_train['user_answer']==item)]), 2)) + '%',
            str(round(100 * len(retained_train[(retained_train['user_answer']==item) & (retained_train['answered_correctly'] == -0)]) / len(retained_train[(retained_train['user_answer']==item)]), 2)) + '%',
            str(round(100 * len(retained_train[(retained_train['user_answer']==item) & (retained_train['answered_correctly'] == 1)]) / len(retained_train[(retained_train['user_answer']==item)]), 2)) + '%',
        ],
        textposition='auto'
    ) for item in retained_train['user_answer'].unique().tolist()
]

for i in range(len(traces)):
    fig.append_trace(traces[i], (i // 2) + 1, (i % 2)  +1)

fig.update_layout(
    title_text='Percent of correct answers for every option',
    height=900,
    width=800
)

fig.show()

Although we observed some kind of imbalancement in terms of numbers, but we can see an almost balanced distribution of percentage over user answers. Let's look at the pair and correlation plot once before we wrap up for this session, sleepy :(.

In [ ]:
retained_train["prior_question_had_explanation"] = retained_train['prior_question_had_explanation'].replace({False:0, True:1})
retained_train

In [ ]:
sns.pairplot(retained_train.drop(columns=["prior_question_had_explanation"]), hue="answered_correctly")
plt.show()

In [ ]:
corr = retained_train.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(10, 10))
    ax = sns.heatmap(corr,mask=mask,square=True,linewidths=.8,cmap="viridis",annot=True)

Okay, `task_container_id` is correlated with `timestamp`. Referring to our previous observations, users with mutlitasking instances i.e. having multiple interactions over single timestamp have same `task_container_id` as well as `prior_question_elapsed_time`. We will see more about this.

In [ ]:
retained_train[["user_id", "timestamp"]].groupby("user_id").shift()

In [ ]:
retained_train[["user_id", "prior_question_elapsed_time"]].groupby("user_id").shift()

# <h2 style="color:dodgerblue">3.4. prior_question_had_explanation stats per high-cardinal features</h2>

In [ ]:
context_explanation_agg = retained_train[["content_id", "prior_question_had_explanation", target]].groupby(["content_id", "prior_question_had_explanation"]).agg("mean")
context_explanation_agg = context_explanation_agg.unstack().reset_index()
context_explanation_agg.columns = ["content_id", "false_mean_explanation", "true_mean_explanation"]
# change the dtypes to save more memory
context_explanation_agg.content_id = context_explanation_agg.content_id.astype("int16")
context_explanation_agg.false_mean_explanation = context_explanation_agg.false_mean_explanation.astype("float16")
context_explanation_agg.true_mean_explanation = context_explanation_agg.content_id.astype("float16")

display(context_explanation_agg)

# <h2 style="color:dodgerblue">3.5. Analysing (and creating) Attempts per categorical features</h2>

In [ ]:
#TBD
#retained_train["attempt_no"] = 1
#retained_train.groupby(["user_id"])["attempt_no"].agg("sum")

In [ ]:
train

# 4. <a id="quest-data">Explore Questions Data</a>

**Metadata for the questions posed to users.**

- `question_id`: foreign key for the train/test content_id column, when the content type is question (0).

- `bundle_id`: code for which questions are served together.

- `correct_answer`: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

- `part`: the relevant section of the TOEIC test.

- `tags`: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [ ]:
questions = pd.read_csv("../input/riiid-test-answer-prediction/questions.csv")
questions

# 5. <a id="lect-data">Explore Lectures Data</a>

**Metadata for the lectures watched by users as they progress in their education.**

- `lecture_id`: foreign key for the train/test content_id column, when the content type is lecture (1).

- `part`: top level category code for the lecture.

- `tag`: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

- `type_of`: brief description of the core purpose of the lecture

In [ ]:
lectures = pd.read_csv("../input/riiid-test-answer-prediction/lectures.csv")
lectures

As described earlier, `lecture_id` is the same as the `content_id` for `content_type_id==1`, let's have a look :)

In [ ]:
train[train['content_type_id']==1].sort_values(['content_id'], ascending=True)

OKay, we will merge the datasets and observe later. But for now, let's explore `tag`, `part` and `type_of`.
As far as `tag` is concerned, the description doesn't speaj much, but it has mentioned that we can cluster the lectures. Let's see.

In [ ]:
lectures.nunique().to_frame().rename(columns={0:"Value-counts"}).style.background_gradient(cmap=colorMap)

So the `lectures.csv` contains 418 unique lecture IDs, 151 unique tags, 7 unique parts and 4 types of lectures. Let's see what's there in `tag`.

In [ ]:
plt.figure(figsize=(30, 10))
sns.barplot(lectures['tag'].value_counts().index, lectures['tag'].value_counts().values,alpha=0.8)
plt.show()

Let's analyse top-15 and buttom-15 tags by count and observe the impact and distribution of `part` and `type_of`.

In [ ]:
top_15_tags = lectures['tag'].value_counts()[:15].index.tolist()
btm_15_tags = lectures['tag'].value_counts()[-15:].index.tolist()

In [ ]:
distribution_plot(lectures, "tag", "blue", 15)

In [ ]:
distribution_plot(lectures, "part", "green", 15)

In [ ]:
distribution_plot(lectures, "type_of", "red", 15)

In [ ]:
topTags = lectures[lectures['tag'].isin(top_15_tags)]
btmTags = lectures[lectures['tag'].isin(btm_15_tags)]

distribution_plot(topTags, "part", "blue", 15)

In [ ]:
distribution_plot(topTags, "type_of", "blue", 15)

In [ ]:
distribution_plot(btmTags, "part", "blue", 15)

In [ ]:
distribution_plot(btmTags, "type_of", "blue", 15)

Tags with lesser counts only have concept type. 

# 6. <a id="ss-sub">Sample Test and Submission Data</a>

In [ ]:
test = pd.read_csv("../input/riiid-test-answer-prediction/example_test.csv")
test

In [ ]:
sub = pd.read_csv("../input/riiid-test-answer-prediction/example_sample_submission.csv")
sub

# Work in Progress :)